<a href="https://colab.research.google.com/github/Pramsi/Can-t_tame_that_Python_Semester_Project/blob/Shap/Can't_tame_that_Python_Semester_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [1]:
#for data manipulation
import numpy as np
import pandas as pd

#for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

#Other tooling
import os

#NLP
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
!pip install --upgrade emoji
import emoji
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

#Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#Explainable ML
!pip install lime
from lime.lime_text import LimeTextExplainer
from sklearn.pipeline import make_pipeline
!pip install shap
import shap

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#Import Datafiles

In [2]:
# create kaggle directory and add kaggle.json
from google.colab import files
files.upload()

! mkdir -p ~/.kaggle #create kaggle directory
! mv kaggle.json ~/.kaggle/ #move json file there
! chmod 600 ~/.kaggle/kaggle.json #change permission of the file

Saving kaggle.json to kaggle.json


In [3]:
#create gitignore and add kaggle
%%writefile .gitignore
.kaggle/

Overwriting .gitignore


In [ ]:
#upload brexit dataset
!kaggle datasets download -d visalakshiiyer/twitter-data-brexit
! mkdir brexitTweets #create folder with files
! unzip twitter-data-brexit.zip -d brexitTweets #unzip data and save in folder

twitter-data-brexit.zip: Skipping, found more recently modified local copy (use --force to force download)
mkdir: cannot create directory ‘brexitTweets’: File exists
Archive:  twitter-data-brexit.zip
replace brexitTweets/TweetDataset_AntiBrexit_Jan-Mar2022.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!ls -a


In [ ]:
#Load Datasets
dfAnti = pd.read_csv("brexitTweets/TweetDataset_AntiBrexit_Jan-Mar2022.csv")
dfPro = pd.read_csv("brexitTweets/TweetDataset_ProBrexit_Jan-Mar2022.csv")

In [ ]:
#Check Data
dfAnti.head()

#Preprocessing

In [ ]:
# label pro and anti tweets and concatenate the two files
dfAnti['Position'] = 0
dfPro['Position'] = 1

df = pd.concat([dfAnti, dfPro])

#drop unwanted data
df.drop(['Unnamed: 0', 'Date', 'Headline', 'URL', 'Opening Text', 'Source', 'Influencer', 'Country', 'Subregion', 'Language', 'Reach', 'Desktop Reach', 'Mobile Reach', 'Twitter Social Echo', 'Facebook Social Echo', 'Reddit Social Echo', 'National Viewership', 'Engagement', 'AVE', 'Key Phrases', 'Input Name', 'Keywords', 'Twitter Authority', 'Tweet Id', 'Twitter Id', 'Twitter Client', 'Twitter Screen Name', 'User Profile Url', 'Twitter Bio', 'Twitter Followers', 'Twitter Following', 'Alternate Date Format', 'Time', 'State', 'City', 'Document Tags'], axis = 1, inplace=True)

#count the data
df.Position.value_counts()

In [ ]:
#Clean Hit Sentence - Remove Retweets
df_no_rt = df[df['Hit Sentence'].str.contains("RT @")==False]
df_no_qt = df_no_rt[df_no_rt['Hit Sentence'].str.contains("QT @")==False]

#how many tweets were retweeted and qoted?
print(f'Amount of retweets and quote tweets in percentage:{round(100*(1-len(df_no_qt)/len(df)),1)}%')
print(df_no_qt)

In [ ]:
#How many Pro and Anti Tweets are left
df_no_qt.Position.value_counts()

In [ ]:
#Preprocess Helper Functions

#to lower case
def lowercase(tweet: str) -> str:
  preprocessedTweet = str.lower(tweet)
  return preprocessedTweet

#remove hashtags
def remove_hashtags(tweet: str) -> str:
  preprocessedTweet = re.sub('#[a-zA-Z0-9_]+','',tweet)
  return preprocessedTweet

#remove mentions
def remove_mentions(tweet: str) -> str:
  preprocessedTweet = re.sub('@[a-zA-Z0-9_]+','',tweet)
  return preprocessedTweet

#remove emojis
def remove_emojis(tweet: str) -> str:
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F700-\U0001F77F"  # alchemical symbols
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub('', tweet)

#remove stopwords
def remove_stopwords(tweet: str) -> str:
    word_list = tweet.split()
    filtered_words = [word for word in word_list if word not in set(stopwords.words('english'))]
    return ' '.join(filtered_words)

# Function to combine all preprocessing steps
def preprocess_tweet(tweet: str) -> str:
    tweet = lowercase(tweet)
    tweet = remove_hashtags(tweet)
    tweet = remove_mentions(tweet)
    tweet = remove_stopwords(tweet)
    tweet = remove_emojis(tweet)
    return tweet

In [ ]:
# no retweets and no quote tweets / no cleaned column yet
df_no_qt.head()

In [ ]:
# create cleaned tweet column
df_no_qt['tweet_clean'] = df_no_qt['Hit Sentence'].apply(preprocess_tweet)

In [ ]:
# with cleaned tweet column
df_no_qt.head()

#Split and Vectorize the data

In [ ]:
# Set X and y label
X_preprocessed = df_no_qt["tweet_clean"]
y = df_no_qt["Sentiment"]

In [ ]:
# Encode the y labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
#check encoded labels
for index, original_label in enumerate(label_encoder.classes_):
    print(f"Encoded Label: {index} -> Original Label: {original_label}")

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y_encoded, test_size=0.3, random_state=42)

In [ ]:
print('X_train shape: ', X_train.shape)
print('X_test shape: ', X_test.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ',y_test.shape)

In [ ]:
#Vectorize the data
tfidf_vectorizer = TfidfVectorizer(stop_words='english') #make sure that all stopwords are excluded, stopword list might differ
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

count_vectorizer = CountVectorizer(stop_words="english") #make sure that all stopwords are excluded
count_train = count_vectorizer.fit_transform(X_train)
count_test = count_vectorizer.transform(X_test)

#Train Models with Tfidf Vectorizer

In [ ]:
# Initialize a Support Vector Classifier (SVC) with a linear kernel
# svm_linear_tfidf = SVC(kernel='linear', C=1)

# Train the classifier using the tfidf training data and training labels
# svm_linear_tfidf.fit(tfidf_train, y_train)

In [ ]:
# Initialize a Logistic Regression Model
log_reg_tfidf = LogisticRegression(C=1, solver='lbfgs', max_iter=1000)

#Train the Linear Regression Model using the tfidf data
log_reg_tfidf.fit(tfidf_train, y_train)

In [ ]:
# Initialize a Random Forest Classifier
random_forest_tfidf = RandomForestClassifier(max_depth=None, n_estimators=100, random_state=42)

# Fit the Random Forest Classifier on the training data
random_forest_tfidf.fit(tfidf_train, y_train)

In [ ]:
# Initialize Naive Bayes Classifier
nb_classifier_tfidf = MultinomialNB()

# Fit the Naive Bayes Classifier with the tfidf data
nb_classifier_tfidf.fit(tfidf_train, y_train)

Make predictions with Tfidf vectorized data

In [ ]:
# Predict the labels for the tfidf vectorized data using a linear SVM model
# y_pred_svm_tfidf = svm_linear_tfidf.predict(tfidf_test)

# Predict the labels for the tfidf vectorized data using a logistic regression model
y_pred_logreg_tfidf = log_reg_tfidf.predict(tfidf_test)

# Predict the labels for the tfidf vectorized data using a Random Forest Classifier
y_pred_forest_tfidf = random_forest_tfidf.predict(tfidf_test)

# Predict the labels for the tfidf vectorized data using a Naive Bayes Classifier
y_pred_nb_tfidf = nb_classifier_tfidf.predict(tfidf_test)

Evaluate our Models with the Tfidf vectorized data

In [ ]:
#SVM Evaluation
# print(f"SVM Evaluation:")
# Calculate and print the accuracy score
# accuracy_svm_tfidf = accuracy_score(y_test, y_pred_svm_tfidf)
# print(f"Accuracy: {accuracy_svm_tfidf}")

#  Calculate and print the F1 score
# f1_svm_tfidf = f1_score(y_test, y_pred_svm_tfidf, average='macro')
# print(f"F1 Score: {f1_svm_tfidf}")

#  Calculate and print the precision score
# precision_svm_tfidf = precision_score(y_test, y_pred_svm_tfidf, average='macro')
# print(f"Precision: {precision_svm_tfidf}")

#  Calculate and print the recall score
# recall_svm_tfidf = recall_score(y_test, y_pred_svm_tfidf, average='macro')
# print(f"Recall: {recall_svm_tfidf}")

# print(f"---------------------------------------------")

print(f"Logistic Regression Evaluation:")
#Logistic Regression Evaluation
# Calculate and print the accuracy score
accuracy_logreg_tfidf = accuracy_score(y_test, y_pred_logreg_tfidf)
print(f"Accuracy: {accuracy_logreg_tfidf}")

# Calculate and print the F1 score
f1_logreg_tfidf = f1_score(y_test, y_pred_logreg_tfidf, average='macro')
print(f"F1 Score: {f1_logreg_tfidf}")

# Calculate and print the precision score
precision_logreg_tfidf = precision_score(y_test, y_pred_logreg_tfidf, average='macro')
print(f"Precision: {precision_logreg_tfidf}")

# Calculate and print the recall score
recall_logreg_tfidf = recall_score(y_test, y_pred_logreg_tfidf, average='macro')
print(f"Recall: {recall_logreg_tfidf}")

print(f"---------------------------------------------")

print(f"Random Forest Evaluation:")
#Random Forest Evaluation
# Calculate and print the accuracy score
accuracy_forest_tfidf = accuracy_score(y_test, y_pred_forest_tfidf)
print(f"Accuracy: {accuracy_forest_tfidf}")

# Calculate and print the F1 score
f1_forest_tfidf = f1_score(y_test, y_pred_forest_tfidf, average='macro')
print(f"F1 Score: {f1_forest_tfidf}")

# Calculate and print the precision score
precision_forest_tfidf = precision_score(y_test, y_pred_forest_tfidf, average='macro')
print(f"Precision: {precision_forest_tfidf}")

# Calculate and print the recall score
recall_forest_tfidf = recall_score(y_test, y_pred_forest_tfidf, average='macro')
print(f"Recall: {recall_forest_tfidf}")

print(f"---------------------------------------------")

print(f"Naive Bayes Classifier Evaluation:")
#Logistic Regression Evaluation
# Calculate and print the accuracy score
accuracy_nb_tfidf = accuracy_score(y_test, y_pred_nb_tfidf)
print(f"Accuracy: {accuracy_nb_tfidf}")

# Calculate and print the F1 score
f1_nb_tfidf = f1_score(y_test, y_pred_nb_tfidf, average='macro')
print(f"F1 Score: {f1_nb_tfidf}")

# Calculate and print the precision score
precision_nb_tfidf = precision_score(y_test, y_pred_nb_tfidf, average='macro', zero_division=1)
print(f"Precision: {precision_nb_tfidf}")

# Calculate and print the recall score
recall_nb_tfidf = recall_score(y_test, y_pred_nb_tfidf, average='macro', zero_division=1)
print(f"Recall: {recall_nb_tfidf}")

print(f"---------------------------------------------")

#Train Models with Count Vectorizer

In [ ]:
# Initialize a Support Vector Classifier (SVC) with a linear kernel
# svm_linear_count = SVC(kernel='linear', C=1)

# Train the classifier using the tfidf training data and training labels
# svm_linear_count.fit(count_train, y_train)

In [ ]:
# Initialize a Logistic Regression Model
log_reg_count = LogisticRegression(C=1, solver='lbfgs', max_iter=1000)

#Train the Logistiv Regression Model using the tfidf data
log_reg_count.fit(count_train, y_train)

In [ ]:
# Initialize a Random Forest Classifier
random_forest_count = RandomForestClassifier(max_depth=None, n_estimators=100, random_state=42)

# Fit the Random Forest Classifier on the training data
random_forest_count.fit(count_train, y_train)

In [ ]:
# Initialize Naive Bayes Classifier
nb_classifier_count = MultinomialNB()

# Fit the Naive Bayes Classifier with the tfidf data
nb_classifier_count.fit(count_train, y_train)

Make predictions with Count vectorized data

In [ ]:
# Predict the labels for the count vectorized data using a linear SVM model
# y_pred_svm_count = svm_linear_count.predict(count_test)

# Predict the labels for the count vectorized data using a logistic regression model
y_pred_logreg_count = log_reg_count.predict(count_test)

# Predict the labels for the count vectorized data using a Random Forest Classifier
y_pred_forest_count = random_forest_count.predict(count_test)

# Predict the labels for the count vectorized data using a Naive Bayes Classifier
y_pred_nb_count = nb_classifier_count.predict(count_test)

Evaluate our Models with the Count vectorized data

In [ ]:
#SVM Evaluation
# print(f"SVM Evaluation:")
# Calculate and print the accuracy score
# accuracy_svm_count = accuracy_score(y_test, y_pred_svm_count)
# print(f"Accuracy: {accuracy_svm_count}")

# Calculate and print the F1 score
# f1_svm_count = f1_score(y_test, y_pred_svm_count, average='macro')
# print(f"F1 Score: {f1_svm_count}")

# Calculate and print the precision score
# precision_svm_count = precision_score(y_test, y_pred_svm_count, average='macro')
# print(f"Precision: {precision_svm_count}")

# Calculate and print the recall score
# recall_svm_count = recall_score(y_test, y_pred_svm_count, average='macro')
# print(f"Recall: {recall_svm_count}")

# print(f"---------------------------------------------")

print(f"Logistic Regression Evaluation:")
#Logistic Regression Evaluation
# Calculate and print the accuracy score
accuracy_logreg_count = accuracy_score(y_test, y_pred_logreg_count)
print(f"Accuracy: {accuracy_logreg_count}")

# Calculate and print the F1 score
f1_logreg_count = f1_score(y_test, y_pred_logreg_count, average='macro')
print(f"F1 Score: {f1_logreg_count}")

# Calculate and print the precision score
precision_logreg_count = precision_score(y_test, y_pred_logreg_count, average='macro')
print(f"Precision: {precision_logreg_count}")

# Calculate and print the recall score
recall_logreg_count = recall_score(y_test, y_pred_logreg_count, average='macro')
print(f"Recall: {recall_logreg_count}")

print(f"---------------------------------------------")

print(f"Random Forest Evaluation:")
#Random Forest Evaluation
# Calculate and print the accuracy score
accuracy_forest_count = accuracy_score(y_test, y_pred_forest_count)
print(f"Accuracy: {accuracy_forest_count}")

# Calculate and print the F1 score
f1_forest_count = f1_score(y_test, y_pred_forest_count, average='macro')
print(f"F1 Score: {f1_forest_count}")

# Calculate and print the precision score
precision_forest_count = precision_score(y_test, y_pred_forest_count, average='macro')
print(f"Precision: {precision_forest_count}")

# Calculate and print the recall score
recall_forest_count = recall_score(y_test, y_pred_forest_count, average='macro')
print(f"Recall: {recall_forest_count}")

print(f"---------------------------------------------")

print(f"Naive Bayes Classifier Evaluation:")
#Logistic Regression Evaluation
# Calculate and print the accuracy score
accuracy_nb_count = accuracy_score(y_test, y_pred_nb_count)
print(f"Accuracy: {accuracy_nb_count}")

# Calculate and print the F1 score
f1_nb_count = f1_score(y_test, y_pred_nb_count, average='macro')
print(f"F1 Score: {f1_nb_count}")

# Calculate and print the precision score
precision_nb_count = precision_score(y_test, y_pred_nb_count, average='macro', zero_division=1)
print(f"Precision: {precision_nb_count}")

# Calculate and print the recall score
recall_nb_count = recall_score(y_test, y_pred_nb_count, average='macro', zero_division=1)
print(f"Recall: {recall_nb_count}")

print(f"---------------------------------------------")

In [ ]:
# parameters_log = {'C': [0.1, 1, 10, 100]}

# # Initialize a Logistic Regression Classifier
# log_reg_count = LogisticRegression(C=1, solver='lbfgs', max_iter=3000)

# # Set up GridSearchCV with the SVC, parameter grid, 5-fold cross-validation, and F1 scoring
# grid_log_reg_count = GridSearchCV(log_reg_count, parameters_log, cv=5, scoring="f1_macro")

# # Train the GridSearchCV on the scaled training data
# grid_log_reg_count.fit(count_train, y_train)

In [ ]:
# print(f"Best Parameters: {grid_log_reg_count.best_params_}")
# print(f"Best F1 - Score: {grid_log_reg_count.best_score_}")

# Data Visualization

## Confusion Matrix

In [ ]:
def plot_conf_matrix_multiclass(conf_mat, classifier_name, original_label):
    #Normalize to get percentages
    conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]

    # Set the size of the figure
    plt.figure(figsize=(10,7))

    # seaborn to create a heatmap of the confusion matrix
    sns.heatmap(conf_mat_normalized, annot=True, fmt='.2%', cmap='Blues',
                xticklabels=original_label, yticklabels=original_label)

    # Label the x-axis as 'Predicted'
    plt.xlabel('Predicted')

    # Label the y-axis as 'Actual'
    plt.ylabel('Actual')

    # Set the title of the plot
    plt.title(f'Confusion Matrix for {classifier_name}')

    # Display the plot
    plt.show()

In [ ]:
conf_matrix_logreg_count_multiclass = confusion_matrix(y_test, y_pred_logreg_count)

In [ ]:
original_label = ["Negative", "Neutral", "Not Rated", "Positive"]
plot_conf_matrix_multiclass(conf_matrix_logreg_count_multiclass, "Logistic Regression with Count Vectorizer", original_label)

## Total counts of sentiments

In [ ]:
#Counting the labels for each sentiment in y_test
neg_y_test = 0
neu_y_test = 0
not_rated_y_test = 0
pos_y_test = 0

for item in y_test:
    if item == 0:
        neg_y_test += 1
    elif item == 1:
        neu_y_test += 1
    elif item == 2:
        not_rated_y_test += 1
    elif item == 3:
        pos_y_test += 1

print("Negative:", neg_y_test)
print("Neutral:", neu_y_test)
print("Not Rated:", not_rated_y_test)
print("Positive:", pos_y_test)

print(f"---------------------------------------------")
#Counting the labels for each sentiment in y_pred_logreg_count
neg_y_pred_logreg_count = 0
neu_y_pred_logreg_count = 0
not_rated_y_pred_logreg_count = 0
pos_y_pred_logreg_count = 0

for item in y_pred_logreg_count:
    if item == 0:
        neg_y_pred_logreg_count += 1
    elif item == 1:
        neu_y_pred_logreg_count += 1
    elif item == 2:
        not_rated_y_pred_logreg_count += 1
    elif item == 3:
        pos_y_pred_logreg_count += 1

print("Negative:", neg_y_pred_logreg_count)
print("Neutral:", neu_y_pred_logreg_count)
print("Not Rated:", not_rated_y_pred_logreg_count)
print("Positive:", pos_y_pred_logreg_count)

In [ ]:
#Data from y_test
y_test_counts = [neg_y_test, neu_y_test, not_rated_y_test, pos_y_test]

#Data from y_pred_logreg_count
y_pred_counts = [neg_y_pred_logreg_count, neu_y_pred_logreg_count, not_rated_y_pred_logreg_count, pos_y_pred_logreg_count]

#Bar chart settings
bar_width = 0.35
index = range(len(y_test_counts))

#Creating the bar chart
plt.bar(index, y_test_counts, bar_width, label='y_test')
plt.bar([i + bar_width for i in index], y_pred_counts, bar_width, label='y_pred_logreg_count')

#Labels, title, and legend
plt.xlabel('Sentiments')
plt.ylabel('Counts')
plt.title('Comparison of Total Sentiment Counts')
plt.xticks([i + bar_width/2 for i in index], ['Negative', 'Neutral', 'Not Rated', 'Positive'])
plt.legend()

#Displaying the plot
plt.tight_layout()
plt.show()

## Split into Pro and Anti Brexit

In [ ]:
#Splitting the dataframe into Pro and Anti Brexit
pro_brexit_df_no_qt = df_no_qt[df_no_qt['Position'] == 1]
anti_brexit_df_no_qt = df_no_qt[df_no_qt['Position'] == 0]

#Creating Dataframes out of that data
df_proBrexit = pd.DataFrame(pro_brexit_df_no_qt)
df_antiBrexit = pd.DataFrame(anti_brexit_df_no_qt)

#Displaying the resulting DataFrame
df_proBrexit.info()
df_antiBrexit.info()

In [ ]:
#Creating a new dataframe with the testing data and the predictions
data = {'X_test': X_test, 'y_test': y_test, 'y_pred_logreg_count': y_pred_logreg_count}
df_new = pd.DataFrame(data)

#Displaying the content
df_new.head()
df_new.info()


In [ ]:
#Merging df_new with pro_brexit_df_no_qt based on common values in 'X_test' and 'tweet_clean'
merged_pro_brexit = pd.merge(df_new, pro_brexit_df_no_qt,how = 'inner', left_on='X_test', right_on='tweet_clean')
df_merged_pro_brexit = pd.DataFrame(merged_pro_brexit)


#Merging df_new with anti_brexit_df_no_qt based on common values in 'X_test' and 'tweet_clean'
merged_anti_brexit = pd.merge(df_new, anti_brexit_df_no_qt, how = 'inner',left_on='X_test', right_on='tweet_clean')
df_merged_anti_brexit = pd.DataFrame(merged_anti_brexit)


#Displaying the duplicated entries
print(df_merged_pro_brexit['X_test'].duplicated().sum())
print(df_merged_pro_brexit['tweet_clean'].duplicated().sum())

print(df_merged_anti_brexit['X_test'].duplicated().sum())
print(df_merged_anti_brexit['tweet_clean'].duplicated().sum())


#Dropping the duplicates
df_merged_pro_brexit = df_merged_pro_brexit.drop_duplicates()
df_merged_anti_brexit = df_merged_anti_brexit.drop_duplicates()

#Displaying the resulting merged DataFrames
print("Merged Pro-Brexit DataFrame:")
df_merged_pro_brexit.info()

print("\nMerged Anti-Brexit DataFrame:")
df_merged_anti_brexit.info()


### Pro Brexit

In [ ]:
#Getting the data we need from the merged pro-brexit dataframe
pro_brexit_y_pred = df_merged_pro_brexit['y_pred_logreg_count']
pro_brexit_y_test = df_merged_pro_brexit['Sentiment']

#Counting the real sentiment for pro Brexit
pro_brexit_neg_y_test = 0
pro_brexit_neu_y_test = 0
pro_brexit_not_rated_y_test = 0
pro_brexit_pos_y_test = 0

for item in pro_brexit_y_test:
    if item == "Negative":
        pro_brexit_neg_y_test += 1
    elif item == "Neutral":
        pro_brexit_neu_y_test += 1
    elif item == "Not Rated":
        pro_brexit_not_rated_y_test += 1
    elif item == "Positive":
        pro_brexit_pos_y_test += 1

#Displaying the results
print("Negative:", pro_brexit_neg_y_test)
print("Neutral:", pro_brexit_neu_y_test)
print("Not Rated:", pro_brexit_not_rated_y_test)
print("Positive:", pro_brexit_pos_y_test)
print(f"---------------------------------------------")

#Counting the predicted sentiment for pro Brexit
pro_brexit_neg_y_pred_logreg_count = 0
pro_brexit_neu_y_pred_logreg_count = 0
pro_brexit_not_rated_y_pred_logreg_count = 0
pro_brexit_pos_y_pred_logreg_count = 0

for item in pro_brexit_y_pred:
    if item == 0:
        pro_brexit_neg_y_pred_logreg_count += 1
    elif item == 1:
        pro_brexit_neu_y_pred_logreg_count += 1
    elif item == 2:
        pro_brexit_not_rated_y_pred_logreg_count += 1
    elif item == 3:
        pro_brexit_pos_y_pred_logreg_count += 1

#Displaying the results
print("Negative:", pro_brexit_neg_y_pred_logreg_count)
print("Neutral:", pro_brexit_neu_y_pred_logreg_count)
print("Not Rated:", pro_brexit_not_rated_y_pred_logreg_count)
print("Positive:", pro_brexit_pos_y_pred_logreg_count)

In [ ]:
# Data from real sentiment
y_test_counts = [pro_brexit_neg_y_test, pro_brexit_neu_y_test, pro_brexit_not_rated_y_test, pro_brexit_pos_y_test]

# Data from y_pred_logreg_count
y_pred_counts = [pro_brexit_neg_y_pred_logreg_count, pro_brexit_neu_y_pred_logreg_count, pro_brexit_not_rated_y_pred_logreg_count, pro_brexit_pos_y_pred_logreg_count]

# Bar chart settings
bar_width = 0.35
index = range(len(y_test_counts))

# Creating the bar chart
plt.bar(index, y_test_counts, bar_width, label='y_test')
plt.bar([i + bar_width for i in index], y_pred_counts, bar_width, label='y_pred_logreg_count')

# Labels, title, and legend
plt.xlabel('Sentiments')
plt.ylabel('Counts')
plt.title('Comparison of Sentiment Counts for Pro-Brexit accounts')
plt.xticks([i + bar_width/2 for i in index], ['Negative', 'Neutral', 'Not Rated', 'Positive'])
plt.legend()

# Displaying the plot
plt.tight_layout()
plt.show()

### Anti Brexit

In [ ]:
anti_brexit_y_pred = df_merged_anti_brexit['y_pred_logreg_count']
anti_brexit_y_test = df_merged_anti_brexit['Sentiment']

#Counting the real sentiment for pro Brexit
anti_brexit_neg_y_test = 0
anti_brexit_neu_y_test = 0
anti_brexit_not_rated_y_test = 0
anti_brexit_pos_y_test = 0

for item in anti_brexit_y_test:
    if item == "Negative":
        anti_brexit_neg_y_test += 1
    elif item == "Neutral":
        anti_brexit_neu_y_test += 1
    elif item == "Not Rated":
        anti_brexit_not_rated_y_test += 1
    elif item == "Positive":
        anti_brexit_pos_y_test += 1

print("REAL SENTIMENT")
print("Negative:", anti_brexit_neg_y_test)
print("Neutral:", anti_brexit_neu_y_test)
print("Not Rated:", anti_brexit_not_rated_y_test)
print("Positive:", anti_brexit_pos_y_test)
print(f"---------------------------------------------")


#Counting the labels for each sentiment in y_pred_logreg_count
anti_brexit_neg_y_pred_logreg_count = 0
anti_brexit_neu_y_pred_logreg_count = 0
anti_brexit_not_rated_y_pred_logreg_count = 0
anti_brexit_pos_y_pred_logreg_count = 0

for item in anti_brexit_y_pred:
    if item == 0:
        anti_brexit_neg_y_pred_logreg_count += 1
    elif item == 1:
        anti_brexit_neu_y_pred_logreg_count += 1
    elif item == 2:
        anti_brexit_not_rated_y_pred_logreg_count += 1
    elif item == 3:
        anti_brexit_pos_y_pred_logreg_count += 1

print("PREDICTED SENTIMENT")
print("Negative:", anti_brexit_neg_y_pred_logreg_count)
print("Neutral:", anti_brexit_neu_y_pred_logreg_count)
print("Not Rated:", anti_brexit_not_rated_y_pred_logreg_count)
print("Positive:", anti_brexit_pos_y_pred_logreg_count)

In [ ]:
# Data from
y_test_counts = [anti_brexit_neg_y_test, anti_brexit_neu_y_test, anti_brexit_not_rated_y_test, anti_brexit_pos_y_test]

# Data from y_pred_logreg_count
y_pred_counts = [anti_brexit_neg_y_pred_logreg_count, anti_brexit_neu_y_pred_logreg_count, anti_brexit_not_rated_y_pred_logreg_count, anti_brexit_pos_y_pred_logreg_count]

# Bar chart settings
bar_width = 0.35
index = range(len(y_test_counts))

# Creating the bar chart
plt.bar(index, y_test_counts, bar_width, label='y_test')
plt.bar([i + bar_width for i in index], y_pred_counts, bar_width, label='y_pred_logreg_count')

# Labels, title, and legend
plt.xlabel('Sentiments')
plt.ylabel('Counts')
plt.title('Comparison of Sentiment Counts for Anti-Brexit accounts')
plt.xticks([i + bar_width/2 for i in index], ['Negative', 'Neutral', 'Not Rated', 'Positive'])
plt.legend()

# Displaying the plot
plt.tight_layout()
plt.show()

# Comparing Results in Percentages

In [ ]:
#Total sum of Pro Brexit labels
total_real_pro_sentiments = pro_brexit_neg_y_test + pro_brexit_neu_y_test + pro_brexit_not_rated_y_test + pro_brexit_pos_y_test
print(total_real_pro_sentiments)


#Total sum of Pro Brexit labels
total_pred_pro_sentiments = pro_brexit_neg_y_pred_logreg_count +pro_brexit_neu_y_pred_logreg_count+ pro_brexit_not_rated_y_pred_logreg_count+ pro_brexit_pos_y_pred_logreg_count
print(total_pred_pro_sentiments)

print(f"---------------------------------------------")

#Total sum of Anti Brexit labels
total_real_anti_sentiments = anti_brexit_neg_y_test + anti_brexit_neu_y_test + anti_brexit_not_rated_y_test + anti_brexit_pos_y_test
print(total_real_anti_sentiments)

#Total sum of Anti Brexit labels
total_pred_anti_sentiments = anti_brexit_neg_y_pred_logreg_count +anti_brexit_neu_y_pred_logreg_count+ anti_brexit_not_rated_y_pred_logreg_count+ anti_brexit_pos_y_pred_logreg_count
print(total_pred_anti_sentiments)

In [ ]:
#Calculate the percentages of each Sentiment

#PRO real
print("PERCENTAGE OF EACH REAL SENTIMENT IN PRO BREXIT")
#Negative
pro_real_percent_of_negative = pro_brexit_neg_y_test / total_real_pro_sentiments * 100
print("Negative")
print(pro_real_percent_of_negative)

#Neutral
pro_real_percent_of_neutral = pro_brexit_neu_y_test / total_real_pro_sentiments * 100
print("Neutral")
print(pro_real_percent_of_neutral)

#Not Rated
pro_real_percent_of_not_rated = pro_brexit_not_rated_y_test / total_real_pro_sentiments * 100
print("Not Rated")
print(pro_real_percent_of_not_rated)

#Positive
pro_real_percent_of_positive = pro_brexit_pos_y_test/ total_real_pro_sentiments * 100
print("Positive")
print(pro_real_percent_of_positive)


print(f"---------------------------------------------")

#PRO predicted

print("PERCENTAGE OF EACH PREDICTED SENTIMENT IN PRO BREXIT")
#Negative
pro_pred_percent_of_negative = pro_brexit_neg_y_pred_logreg_count / total_pred_pro_sentiments * 100
print("Negative")
print(pro_pred_percent_of_negative)

#Neutral
pro_pred_percent_of_neutral = pro_brexit_neu_y_pred_logreg_count / total_pred_pro_sentiments * 100
print("Neutral")
print(pro_pred_percent_of_neutral)

#Not Rated
pro_pred_percent_of_not_rated = pro_brexit_not_rated_y_pred_logreg_count / total_pred_pro_sentiments * 100
print("Not Rated")
print(pro_pred_percent_of_not_rated)

#Positive
pro_pred_percent_of_positive = pro_brexit_pos_y_pred_logreg_count / total_pred_pro_sentiments * 100
print("Positive")
print(pro_pred_percent_of_positive)


In [ ]:
#ANTI real
print("PERCENTAGE OF EACH REAL SENTIMENT IN ANTI BREXIT")
#Negative
anti_real_percent_of_negative = anti_brexit_neg_y_test / total_real_anti_sentiments * 100
print("Negative")
print(anti_real_percent_of_negative)

#Neutral
anti_real_percent_of_neutral = anti_brexit_neu_y_test / total_real_anti_sentiments * 100
print("Neutral")
print(anti_real_percent_of_neutral)

#Not Rated
anti_real_percent_of_not_rated = anti_brexit_not_rated_y_test / total_real_anti_sentiments * 100
print("Not Rated")
print(anti_real_percent_of_not_rated)

#Positive
anti_real_percent_of_positive = anti_brexit_pos_y_test/ total_real_anti_sentiments * 100
print("Positive")
print(anti_real_percent_of_positive)


print(f"---------------------------------------------")



#ANTI predicted

print("PERCENTAGE OF EACH PREDICTED SENTIMENT IN ANTI BREXIT")
#Negative
anti_pred_percent_of_negative = anti_brexit_neg_y_pred_logreg_count / total_pred_anti_sentiments * 100
print("Negative")
print(anti_pred_percent_of_negative)

#Neutral
anti_pred_percent_of_neutral = anti_brexit_neu_y_pred_logreg_count / total_pred_anti_sentiments * 100
print("Neutral")
print(anti_pred_percent_of_neutral)

#Not Rated
anti_pred_percent_of_not_rated = anti_brexit_not_rated_y_pred_logreg_count / total_pred_anti_sentiments * 100
print("Not Rated")
print(anti_pred_percent_of_not_rated)

#Positive
anti_pred_percent_of_positive = anti_brexit_pos_y_pred_logreg_count / total_pred_anti_sentiments * 100
print("Positive")
print(anti_pred_percent_of_positive)


In [ ]:

# Pro Brexit Sentiments
pro_sentiments = ['Negative', 'Neutral', 'Not Rated', 'Positive']
pro_percentages = [pro_pred_percent_of_negative, pro_pred_percent_of_neutral, pro_pred_percent_of_not_rated, pro_pred_percent_of_positive]

plt.figure(figsize=(10, 6))
bars = plt.bar(pro_sentiments, pro_percentages, color=['red', 'orange', 'gray', 'green'])
plt.title('Percentage of Sentiments in Pro-Brexit')
plt.xlabel('Sentiments')
plt.ylabel('Percentage')
plt.ylim(0, 100)  # Set y-axis limit from 0 to 100 for percentages

# Adding labels to each bar
for bar, percentage in zip(bars, pro_percentages):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 1,
             f'{percentage:.2f}%', ha='center', va='bottom')

plt.show()

# Anti Brexit Sentiments
anti_sentiments = ['Negative', 'Neutral', 'Not Rated', 'Positive']
anti_percentages = [anti_pred_percent_of_negative, anti_pred_percent_of_neutral, anti_pred_percent_of_not_rated, anti_pred_percent_of_positive]

plt.figure(figsize=(10, 6))
bars = plt.bar(anti_sentiments, anti_percentages, color=['red', 'orange', 'gray', 'green'])
plt.title('Percentage of Sentiments in Anti-Brexit')
plt.xlabel('Sentiments')
plt.ylabel('Percentage')
plt.ylim(0, 100)  # Set y-axis limit from 0 to 100 for percentages

# Adding labels to each bar
for bar, percentage in zip(bars, anti_percentages):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 1,
             f'{percentage:.2f}%', ha='center', va='bottom')

plt.show()



In [ ]:
#Plotting the data

# Pro-Brexit Sentiments - Real vs Predicted
pro_real_sentiments = ['Negative', 'Neutral', 'Not Rated', 'Positive']
pro_real_percentages = [pro_real_percent_of_negative, pro_real_percent_of_neutral, pro_real_percent_of_not_rated, pro_real_percent_of_positive]
pro_pred_percentages = [pro_pred_percent_of_negative, pro_pred_percent_of_neutral, pro_pred_percent_of_not_rated, pro_pred_percent_of_positive]

plt.figure(figsize=(10, 6))
bar_width = 0.35
index = range(len(pro_real_sentiments))

bars1 = plt.bar(index, pro_real_percentages, bar_width, label='Real', color='blue')
bars2 = plt.bar([i + bar_width for i in index], pro_pred_percentages, bar_width, label='Predicted', color='orange')

plt.xlabel('Sentiments')
plt.ylabel('Percentage')
plt.title('Comparison of Real vs Predicted Sentiments in Pro-Brexit')
plt.xticks([i + bar_width / 2 for i in index], pro_real_sentiments)
plt.legend()

# Adding labels to each bar
for bars in [bars1, bars2]:
    for bar in bars:
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 1,
                 f'{bar.get_height():.2f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Anti-Brexit Sentiments - Real vs Predicted
anti_real_sentiments = ['Negative', 'Neutral', 'Not Rated', 'Positive']
anti_real_percentages = [anti_real_percent_of_negative, anti_real_percent_of_neutral, anti_real_percent_of_not_rated, anti_real_percent_of_positive]
anti_pred_percentages = [anti_pred_percent_of_negative, anti_pred_percent_of_neutral, anti_pred_percent_of_not_rated, anti_pred_percent_of_positive]

plt.figure(figsize=(10, 6))
bars1 = plt.bar(index, anti_real_percentages, bar_width, label='Real', color='blue')
bars2 = plt.bar([i + bar_width for i in index], anti_pred_percentages, bar_width, label='Predicted', color='orange')

plt.xlabel('Sentiments')
plt.ylabel('Percentage')
plt.title('Comparison of Real vs Predicted Sentiments in Anti-Brexit')
plt.xticks([i + bar_width / 2 for i in index], anti_real_sentiments)
plt.legend()

# Adding labels to each bar
for bars in [bars1, bars2]:
    for bar in bars:
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 1,
                 f'{bar.get_height():.2f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()


#Explainability Techniques

###LIME

In [ ]:
# make pipeline in order to reverse vectorized data
model = make_pipeline(CountVectorizer(), LogisticRegression(C=1, solver='lbfgs', max_iter=1000))
model.fit(X_train, y_train)

In [ ]:
# initialize explainer
explainer = LimeTextExplainer(class_names=['Negative', 'Neutral', 'Not Rated', 'Positive'])

In [ ]:
#reset the index, so it starts from 0
X_test.reset_index(drop=True, inplace=True)

#set index and predict probabilities and explain why
idx = 1
exp = explainer.explain_instance(X_test[idx], model.predict_proba, num_features=6)
exp.show_in_notebook()

In [ ]:
#Random Selection of indices

# Define number of instances
num_to_select = 10

# Generate random indices
random_indices = np.random.choice(X_test.shape[0], num_to_select, replace=False)

# Iterate over each randomly selected index
for idx in random_indices:
    # Get the instance to explain
    instance_to_explain = X_test.iloc[[idx]]  # double brackets bc we want to keep the DataFrame structure

    # Get the model's prediction for this instance. This returns probabilities for each class.
    model_prediction = model.predict_proba(instance_to_explain)[0]

    # Get the index of the highest probability (which is the predicted class)
    predicted_class_index = model_prediction.argmax()

    # Explain the instance for the predicted class
    exp = explainer.explain_instance(instance_to_explain.iloc[0], model.predict_proba, num_features=6, labels=[predicted_class_index])
    exp.show_in_notebook()

##SHAP

In [ ]:
#create Shap Explainer with reduced number of samples
# experiment with trade-off between computational time and accuracy --> tried with 1000 (too much for my tiny laptop xD), 100
num_background_samples = 100
explainer = shap.KernelExplainer(log_reg_count.predict_proba, shap.sample(count_train, num_background_samples))

In [ ]:
# Specify the number of instances to explain (adjust as needed)
num_instances_to_explain = 5

# Randomly select indices for the subset of instances (random 100 are chosen of the whole test set)
random_indices = np.random.choice(count_test.shape[0], num_instances_to_explain, replace=False)

# Generate SHAP values for the randomly selected instances
shap_values_subset = explainer.shap_values(count_test[random_indices])

In [ ]:
# Create summary plot for overall feature importance
shap.summary_plot(shap_values_subset, count_test[random_indices], feature_names=count_vectorizer.get_feature_names_out())

In [ ]:
index = 0
count_test_instance = count_test[random_indices[index]]

# Force plot to explain the prediction for a specific instance
shap.force_plot(explainer.expected_value[1], shap_values_subset[1][index], count_test_instance, feature_names=count_vectorizer.get_feature_names_out())